# 自然言語処理
1. Natural Language processing: NLP
1. 方法論
    1. シソーラスによる手法
        1. シソーラス：同じ意味の単語(同義語)・意味の似た単語(類義語)を同じグループに分類
            1. WordNet(プリンストン大学)が有名
    2. カウントベースの手法
        1. コーパスを用いる
            1. コーパスとは目的を持って集められたテキストデータ
            
    3. 推論ベースの手法(word2vec)
    

In [1]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')
    
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
            
    corpus = np.array([word_to_id[w] for w in words])
        
    return corpus, word_to_id, id_to_word

In [2]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus, word_to_id, id_to_word)

[0 1 2 3 4 1 5 6] {'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6} {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


1. 単語の分散表現<br>
    単語の意味を捉えたベクトル表現
1. 単語の分布仮設 (distributional hypothesis)<br>
    自然言語処理において単語をベクトルで表す方法として使われているアイディア。
1. 共起行列<br>
    分布仮設に基づいて単語をベクトルで表す方法
1. ベクトル間の類似度

    コサイン類似度(cosine similarity)を用いる
$$
\begin{align}
similarity(x, y)
= \frac{{\bf x \cdot y}}{{\bf \|x\|\|y\|}} 
= \frac{x_1y_1 + \cdots + x_ny_n}{\sqrt{(x_1^2+ \cdots +x_n^2)}\sqrt{(y_1^2+ \cdots + y_n^2}}
\end{align}
$$

これを素直に実装すると、0ベクトルが引数に来たときに、”0除算”が生じてしまう。これを回避するため、小さな値を除算に加算する。


In [3]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
                
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
                
    return co_matrix

In [4]:
def cos_similarity(x, y, eps=1e-8):
    nx = x / np.sqrt(np.sum(x**2) + eps) # xの正規化
    ny = y / np.sqrt(np.sum(y**2) + eps) # yの正規化
    return np.dot(nx, ny)

In [5]:
text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']] # you の単語ベクトル
c1 = C[word_to_id['i']] # i の単語ベクトル
print(cos_similarity(c0, c1))

0.7071067758832467
